In [27]:
from data_rnn import load_ndfa, load_brackets
from data_prep import pad_and_convert
import pandas as pd
import numpy as np
import torch
from torch import nn

In [28]:
x_train_ndfa, (i2w_ndfa, w2i_ndfa) = load_ndfa(n=150_000)
x_train_brackets, (i2w_brackets, w2i_brackets) = load_brackets(n=150_000)

In [29]:
print(''.join([i2w_ndfa[i] for i in  x_train_ndfa[10_000]]) )

ss


In [30]:
print(''.join([i2w_brackets[i] for i in  x_train_brackets[10_000]]) )

()


In [31]:
len(x_train_ndfa[-1])

158

In [35]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, emb_size, h, num_char, n_layers=1):
        super(LSTM, self).__init__()

        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.lstm = nn.LSTM(input_size=emb_size, hidden_size=h, num_layers=n_layers, batch_first=True)
        self.fc = nn.Linear(h, num_char)
    
    def forward(self, input_seq, h):

        embedded = self.embedding(input_seq)
        lstm_out, hidden = self.lstm(embedded, h)
        output = self.fc(lstm_out)
        return output, hidden


In [33]:
x_train_ndfa = pad_and_convert(x_train_ndfa)
x_train_brackets = pad_and_convert(x_train_brackets)

In [36]:
model = LSTM(vocab_size=len(set(w2i_ndfa)), emb_size=300, h=300, num_char=len(set(w2i_ndfa)), n_layers=1)

In [ ]:
epochs = 1
for epoch in range(epochs):
    model.train()
